In [1]:
#bibliotecas padrao
import numpy as np
import pandas as pd 

#webscrapping
from bs4 import BeautifulSoup
import requests

#plot interativo
import chart_studio.plotly as py 
import chart_studio.tools as tls 
import plotly.graph_objs as go
import cufflinks as cf

#data visualization
import matplotlib.pyplot as plt 
import seaborn as sb 

In [ ]:
#entrar no site chart-studio.plotly.com/ e criar uma conta (se quiser que fique disponível online)
tls.set_credentials_file(username = 'YOUR USER NAME', api_key = 'YOUT API KEY')

In [4]:
df = pd.read_csv('caso.csv')
df

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2021-02-15,AP,NaN,state,80663,1106,329,True,845731.0,861773.0,16.0,9360.12152,0.0137
1,2021-02-14,AP,NaN,state,80528,1102,328,False,845731.0,861773.0,16.0,9344.45614,0.0137
2,2021-02-13,AP,NaN,state,80374,1102,327,False,845731.0,861773.0,16.0,9326.58600,0.0137
3,2021-02-12,AP,NaN,state,80191,1098,326,False,845731.0,861773.0,16.0,9305.35071,0.0137
4,2021-02-11,AP,NaN,state,79835,1093,325,False,845731.0,861773.0,16.0,9264.04053,0.0137
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479389,2020-06-23,SP,Óleo,city,1,0,5,False,2496.0,2471.0,3533809.0,40.46945,0.0000
1479390,2020-06-22,SP,Óleo,city,1,0,4,False,2496.0,2471.0,3533809.0,40.46945,0.0000
1479391,2020-06-21,SP,Óleo,city,1,0,3,False,2496.0,2471.0,3533809.0,40.46945,0.0000
1479392,2020-06-20,SP,Óleo,city,1,0,2,False,2496.0,2471.0,3533809.0,40.46945,0.0000


In [5]:
#dias
len(df.date.unique())

357

In [6]:
#.loc[linhas, colunas]

estados = df.loc[df.place_type == 'state', :]  #pegando apenas os estados
estados.head()

,date,state,city,place_type,confirmed,deaths,order_for_place,is_last,estimated_population_2019,estimated_population,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
0,2021-02-15,AP,NaN,state,80663,1106,329,True,845731.0,861773.0,16.0,9360.12152,0.0137
1,2021-02-14,AP,NaN,state,80528,1102,328,False,845731.0,861773.0,16.0,9344.45614,0.0137
2,2021-02-13,AP,NaN,state,80374,1102,327,False,845731.0,861773.0,16.0,9326.58600,0.0137
3,2021-02-12,AP,NaN,state,80191,1098,326,False,845731.0,861773.0,16.0,9305.35071,0.0137
4,2021-02-11,AP,NaN,state,79835,1093,325,False,845731.0,861773.0,16.0,9264.04053,0.0137


In [7]:
estados.shape

(9077, 13)

In [8]:
todos_os_estados = list(estados.state.unique())

print(todos_os_estados)
print('Total de Estados: ',len(todos_os_estados))

['AP', 'AC', 'AM', 'DF', 'AL', 'CE', 'ES', 'GO', 'MS', 'MA', 'BA', 'MT', 'PA', 'PB', 'MG', 'PE', 'PI', 'PR', 'RR', 'RO', 'RJ', 'RN', 'SE', 'SC', 'TO', 'RS', 'SP']
Total de Estados:  27


In [9]:
data = estados[['date','state','confirmed','deaths','is_last']]  #colunas de interesse
data.head()

,date,state,confirmed,deaths,is_last
0,2021-02-15,AP,80663,1106,True
1,2021-02-14,AP,80528,1102,False
2,2021-02-13,AP,80374,1102,False
3,2021-02-12,AP,80191,1098,False
4,2021-02-11,AP,79835,1093,False


In [10]:
#funcao que gera o dataframe que será plotado
def desdePrimeiroCaso(dados,estados, dias, situacao):
    '''
    type(dados) == DataFrame
    type(estados) == list
    type(dias) == int [0,357]
    type(situacao) == str [confirmed, deaths]
    
    '''

    covid = pd.DataFrame()

    for estado in estados:
        #pegando os estados
        df_estado = data.loc[data.state == estado, :] 
        
        #agrupando por data, somando os com mesma situação e resetando índice
        df_estado = df_estado.groupby('date').sum()[situacao].reset_index()
            
        #dropando coluna date
        df_estado = df_estado.drop('date', axis = 1)
        df_estado.columns = [estado]
        
        #gerando dataFrame covid com os dados de interesse
        covid = pd.concat([covid, df_estado], ignore_index = False, axis = 1)
    
    covid.columns = estados
    
    return covid.head(dias)

In [11]:
df_estado = desdePrimeiroCaso(df, todos_os_estados, 350, 'confirmed')
df_estado

,AP,AC,AM,DF,AL,CE,ES,GO,MS,MA,...,PR,RR,RO,RJ,RN,SE,SC,TO,RS,SP
0,1.0,3.0,2.0,1.0,1.0,9.0,1.0,3.0,2.0,1.0,...,6.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1
1,1.0,3.0,2.0,1.0,1.0,10.0,1.0,3.0,2.0,2.0,...,6.0,2.0,3.0,2.0,1.0,5.0,3.0,1.0,2.0,1
2,1.0,4.0,3.0,1.0,1.0,19.0,1.0,4.0,4.0,2.0,...,6.0,2.0,3.0,3.0,1.0,6.0,5.0,1.0,4.0,1
3,1.0,7.0,7.0,1.0,1.0,24.0,1.0,4.0,6.0,8.0,...,12.0,8.0,5.0,15.0,1.0,7.0,6.0,2.0,4.0,2
4,1.0,11.0,11.0,2.0,1.0,68.0,1.0,9.0,7.0,8.0,...,14.0,8.0,5.0,24.0,6.0,10.0,7.0,5.0,8.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,NaN,NaN,NaN,NaN,NaN,NaN,310763.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1889969
346,NaN,NaN,NaN,NaN,NaN,NaN,311278.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1901574
347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1911411
348,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1913598


In [12]:
df_estado.iplot(filename = 'evolucao-covid-19', 
            title = 'Evolução dos casos confirmados de SarsCov-2 no Brasil por estado (by Tatiane Corrêa)',
           theme = 'pearl',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de casos confirmados',
           world_readable=True)